# Collecting Information about Companies through API(Diffbot and PatentsView)

In [5]:
import pandas as pd
import numpy as np

In [201]:
#load the original input file
original=pd.read_csv("Diffbot.csv")

In [202]:
original

,gvkey,conm,region
0,3164,COLE NATIONAL CORP,Ohio
1,3469,CONTINENTAL RESOURCES INC,Oklahoma
2,3862,AMERICAN CLASSIC VOYAGES CO,Louisiana
3,3905,ADTALEM GLOBAL EDUCATION INC,Illinois
4,4850,FORT HOWARD CORP,Wisconsin
...,...,...,...
3228,264507,JACKSON HEWITT TAX SERVICE,New Jersey
3229,264708,CABELAS INC,Nebraska
3230,264713,INFOSONICS CORP -OLD,California
3231,265008,21ST CENTURY ONCOLOGY HLDGS,Florida


In [204]:
#check na values
print(original[original.region.isnull()])

       gvkey                          conm region
212    22819          BANNER AEROSPACE INC    NaN
490    25117        CELERITY SOLUTIONS INC    NaN
1327   31537        U S OFFICE PRODUCTS CO    NaN
1618   61690                           NaN    NaN
1870   63338       STRATEGIC EDUCATION INC    NaN
2155   64822       CAREY INTERNATIONAL INC    NaN
2279   65593     U S A FLORAL PRODUCTS INC    NaN
2345   65936    BUILDING ONE SERVICES CORP    NaN
2512  113028    CLASSIC VACATION GROUP INC    NaN
2557  118502                       CEB INC    NaN
2584  120633     ARDENT COMMUNICATIONS INC    NaN
2645  124442  XM SATELLITE RADIO HLDGS INC    NaN
2809  145042             ADVISORY BOARD CO    NaN
3149  183228      ZST DIGITAL NETWORKS INC    NaN


In [205]:
#remove na values
original=original.dropna()
original = original.reset_index(drop=True)
print(original)

       gvkey                          conm        region
0       3164            COLE NATIONAL CORP          Ohio
1       3469     CONTINENTAL RESOURCES INC      Oklahoma
2       3862   AMERICAN CLASSIC VOYAGES CO     Louisiana
3       3905  ADTALEM GLOBAL EDUCATION INC      Illinois
4       4850              FORT HOWARD CORP     Wisconsin
...      ...                           ...           ...
3214  264507    JACKSON HEWITT TAX SERVICE    New Jersey
3215  264708                   CABELAS INC      Nebraska
3216  264713          INFOSONICS CORP -OLD    California
3217  265008   21ST CENTURY ONCOLOGY HLDGS       Florida
3218  311524                  TAMINCO CORP  Pennsylvania

[3219 rows x 3 columns]


In [206]:
# check na values
print(original[original.region.isnull()])

Empty DataFrame
Columns: [gvkey, conm, region]
Index: []


In [459]:
# a small testing dataset
original_first100=original.head(100)

In [504]:
#this dataset is generated randomly with both VC and nonVC companies, so it can be used for further comparisons
random_ipo=pd.read_csv("ipo_random.csv")

In [1]:
#a testing sample for further testing the code
original500=random_ipo.head(500)

NameError: name 'random_ipo' is not defined

In [506]:
import time

In [507]:
# a testing sample of one company pulling from API with selected columns

import requests

url = "https://kg.diffbot.com/kg/v3/dql?token="

payload = {
    "type": "query",
    "from": 0,
    "size": 50,
    "format": "csv",
    "nonCanonicalFacts": False,
    "noDedupArticles": False,
    "query": "type:Corporation name:\"CONTINENTAL RESOURCES INC\" location.region.name:”Oklahoma”",
    "exportspec": "name,Name;;homepageUri,Homepage;;ceo.name,CEO;;industries,Industries;;summary,Summary;;nbEmployeesMax,Employees;;location.address,Address;;location.city.name,City;;"
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
v2=response.text
print(v2)


Name,,Homepage,,CEO,,Industries,,Summary,,Employees,,Address,,City,,
Continental Resources Inc.,,clr.com,,William Berry,,Oil Companies,,"Energy company based in Oklahoma City, Oklahoma, United States",,1254,,"20 N. Broadway, Oklahoma City, 73102, Oklahoma, United States",,Oklahoma City,,



In [508]:
# use its headers for our output csv
text1=v2.split(',,')
text1.pop()
text1
Title=text1[0:8]
colname='gvkey'
Title.insert(0,colname)
print(Title)

['gvkey', 'Name', 'Homepage', 'CEO', 'Industries', 'Summary', 'Employees', 'Address', 'City']


In [509]:
# using while loop to extract company information from API(only with selected columns(information))
# this is just for selected columns where the export fields are specified. There will be a chunk of code where all the variables given by the API will be saved to a csv
i=0
import requests
url = "https://kg.diffbot.com/kg/v3/dql?token="

import csv
from csv import writer
with open('500API Information.csv','w') as f:
    write = csv.writer(f) 
    write.writerow(Title) 
    while i<=499:
        if i%10==0:
            time.sleep(5)
        id=original500.iloc[i].gvkey
        company=original500.iloc[i].conm
        state=original500.iloc[i].region
        payload = {
        "type": "query",
        "from": 0,
        "size": 50,
        "format": "csv",
        "nonCanonicalFacts": False,
        "noDedupArticles": False,
        "query": "type:Corporation name:\""+company+"\" location.region.name:”"+state+"”",
        "exportspec": "name,Name;;homepageUri,Homepage;;ceo.name,CEO;;industries,Industries;;summary,Summary;;nbEmployeesMax,Employees;;location.address,Address;;location.city.name,City;;"
        }
        headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
        }
        response = requests.post(url, json=payload, headers=headers)

        temp=response.text
        text=temp.split(',,')
        text.pop()
        Content=text[8:16]
        if not Content:
            name=original500.iloc[i].conm
            Content.append(name)
        Content.insert(0,id)
        write.writerow(Content)
        i+=1

In [510]:
#make changes to previous version: companies with no API info are deleted
tempdf=pd.read_csv("500API Information.csv")
tempdf

,gvkey,Name,Homepage,CEO,Industries,Summary,Employees,Address,City
0,62599,\nLucent,lucent.com,NaN,Telecommunications Companies,"""Former business, merged with Alcatel in 2006""",50000.0,"""600 Mountain Ave 700, New Providence, New Jer...",New Providence
1,187575,\nTILLY S,tillys.com,Edmond Thomas,Clothing Retailers,American retail clothing company,10000.0,"""Irvine, California""",Irvine
2,112095,\nMips,mips.com,NaN,Semiconductor Companies,"""Manufacturing company based in Sunnyvale, Cal...",50.0,"""Sunnyvale, California""",Sunnyvale
3,119894,\nLog On America,NaN,NaN,Data Providers,"""Organization based in Providence, Rhode Islan...",200.0,"""One Cookson, Place 6th Floor, Providence, Rho...",Providence
4,28194,\nAvient Corporation,avient.com,Robert M. Patterson,Petrochemical Companies,"""Company in Avon Lake, United States""",8700.0,"""33587 Walker Road, Avon Lake, 44012, Ohio, Un...",Avon Lake
...,...,...,...,...,...,...,...,...,...
495,111492,PANACOS PHARMACEUTICALS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,165734,\nAventine Renewable Energy,aventinerei.com,Mark Beemer,Oil Companies,"""Bioethanol and biodiesel company based in Pek...",500.0,"""1300 S. Second Street, Pekin, Illinois""",Pekin
497,25959,WILSHIRE TECHNOLOGIES INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,29288,CLINTRIALS RESEARCH INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [511]:
#merge the original dataframe with the dataframe generated by the API, so we can still keep track of all the information
selected500=pd.concat([original500, tempdf], axis=1)

In [512]:
selected500.to_csv('selected500.csv',index=False)

In [75]:
nocom=nocom.dropna(subset=['Name'])

In [83]:
nocom = nocom.reset_index(drop=True)
nocom.rename(columns = {'ID':'gvkey'}, inplace = True)
print(nocom)

       gvkey                           Name  \
0       3164           \nCOLE NATIONAL CORP   
1       3469   \nContinental Resources Inc.   
2       4850             \nFORT HOWARD CORP   
3       5831         \nIEC Electronics Corp   
4       6116  \nINTERNATIONAL SPEEDWAY CORP   
...      ...                            ...   
1926  264387            \nANGIODYNAMICS INC   
1927  264397                  \nCardtronics   
1928  264416    \nULTRA CLEAN HOLDINGS INC.   
1929  264507               \nJackson Hewitt   
1930  264708                  \nCABELAS INC   

                                  Homepage                  CEO  \
0                         colenational.com                  NaN   
1                                  clr.com        William Berry   
2                                      NaN                  NaN   
3                      iec-electronics.com      Jeff Schlarbaum   
4     internationalspeedwaycorporation.com  Lesa France Kennedy   
...                              

In [84]:
#save to a csv file
nocom.to_csv('nocom.csv',index=False)

In [305]:
#check for the max number of columns among the first 100 companies

i=0
import requests
url = "https://kg.diffbot.com/kg/dql_endpoint?token="
import json
length_list=[]
while i<=99:
    company=original_first100.iloc[i].conm
    state=original_first100.iloc[i].region
    payload = {
        "type": "query",
        "from": 0,
        "size": 50,
        "format": "json",
        "nonCanonicalFacts": False,
        "noDedupArticles": False,
        "query": "type:Corporation name:\""+company+"\" location.region.name:”"+state+"”"
        }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
        }
    response = requests.post(url, json=payload, headers=headers)
    temp=response.text
    temp_data=json.loads(temp)
    temp_company_data=temp_data['data']
    count = 0
    for com in temp_company_data:
        if count == 0:
            header = com.keys()
    i+=1
    length_list.append(len(header))
    
print(max(length_list))

68


In [308]:
# For example, I found out that the company called Continental Resources have 68 variables, so I use the corresponding keys in the json output as the header for my dataframe
    payload = {
        "type": "query",
        "from": 0,
        "size": 50,
        "format": "json",
        "nonCanonicalFacts": False,
        "noDedupArticles": False,
        "query": "type:Corporation name:\"CONTINENTAL RESOURCES INC\" location.region.name:”Oklahoma”"
        }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
        }
    response = requests.post(url, json=payload, headers=headers)
    temp=response.text
    temp_data=json.loads(temp)
    temp_company_data=temp_data['data']
    count = 0
    for com in temp_company_data:
        if count == 0:
            header_final = com.keys()
    print(header_final)

dict_keys(['twitterUri', 'nbActiveEmployeeEdges', 'type', 'naceClassification', 'iSicClassification', 'allNames', 'isAcquired', 'revenue', 'competitors', 'allIds_DiffbotUsage', 'yearlyRevenues', 'logo', 'id', 'stock', 'nbOrigins', 'sicClassification', 'foundingDate', 'image', 'images', 'wikipediaUri', 'secForms', 'irsEmployerIdentificationNumbers', 'diffbotUri', 'nbIncomingEdges', 'nbEmployeesMin', 'ipo', 'capitalization', 'name', 'nbEmployeesMax', 'totalInvestment', 'allOriginHashes', 'linkedInUri', 'naicsClassification', 'nbEmployees', 'isDissolved', 'importance', 'origin', 'description', 'homepageUri', 'founders', 'ceo', 'investments', 'acquiredBy', 'phoneNumbers', 'diffbotClassification', 'descriptors', 'partnerships', 'isNonProfit', 'origins', 'isPublic', 'categories', 'crawlTimestamp', 'nbUniqueInvestors', 'facebookUri', 'secCentralIndexKeys', 'summary', 'types', 'technographics', 'boardMembers', 'allUris', 'quarterlyRevenues', 'nbLocations', 'crunchbaseUri', 'industries', 'allDe

In [483]:
#create an empty dataframe for collecting the json output
com_df = pd.DataFrame(columns=header_final)

In [484]:
com_df

,twitterUri,nbActiveEmployeeEdges,type,naceClassification,iSicClassification,allNames,isAcquired,revenue,competitors,allIds_DiffbotUsage,...,boardMembers,allUris,quarterlyRevenues,nbLocations,crunchbaseUri,industries,allDescriptions,location,locations,subsidiaries


In [485]:
#covnert the dict keys to a list so we can access by index
list_header=list(header_final)

In [487]:
display(com_df)

,twitterUri,nbActiveEmployeeEdges,type,naceClassification,iSicClassification,allNames,isAcquired,revenue,competitors,allIds_DiffbotUsage,...,boardMembers,allUris,quarterlyRevenues,nbLocations,crunchbaseUri,industries,allDescriptions,location,locations,subsidiaries


In [488]:
#create an empty row, so we can append to the dataframe if the company is not found in the Diffbot API
empty_row=[]
for x in range(68):
    empty_row.append(np.nan)

In [489]:
len(empty_row)

68

In [494]:
#this is the code for looping through the 500 companies in the testing sample, and it will return a dataframe with 500 rows and 68 columns. Each line represents a company and each column represents a piece of information extracted from the API. 
#I created an empty dataframe previously since the number of columns(piecies of information) is not the same for each company, so, by creating an empty dataframe with the greatest number of variables, we can fill out nan for companies that having missing information for certain columns. 
i=0
import requests
url = "https://kg.diffbot.com/kg/dql_endpoint?token="
import json

while i<=499:#loop through the 500 companies
    company=random_ipo500.iloc[i].conm #store the company name for each loop
    state=random_ipo500.iloc[i].regionn #store the company founding state for each loop
    payload = {
        "type": "query",
        "from": 0,
        "size": 50,
        "format": "json",
        "nonCanonicalFacts": False,
        "noDedupArticles": False,
        "query": "type:Corporation name:\""+company+"\" location.region.name:”"+state+"”"
        }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
        }
    response = requests.post(url, json=payload, headers=headers)
    temp=response.text #save the json output to a variable, type is still string
    temp_data=json.loads(temp) #the output is now converted to type dictionary
    temp_company_data=temp_data['data'] #only the data part is useful for collecting information
    j=0
    com_info=[] #create an empty list to store the information for each company
    if temp_company_data==[]:#if the company is not found in the API, append an empty row to the dataframe instead
        com_info=empty_row
        com_df.loc[len(com_df)]=com_info
    else:
        for com in temp_company_data:
            while j<=67: #loop through the headers in the prior generated dataframe
                temp_header=list_header[j] #store each header to a temporary vairable
                check_value=com.get(temp_header) #check if the header exists in the json output for this company
                if check_value==None: #if it doesn't, fill the cell with nan
                    com_info.append(np.nan)
                else:
                    com_info.append(check_value) # if it does, fill the cell with the corresponding value
                j+=1 #loop through the dataset headers
    
        com_df.loc[len(com_df)] = com_info #append the entire list of company information to the previously created dataframe
    i+=1

In [495]:
com_df

,twitterUri,nbActiveEmployeeEdges,type,naceClassification,iSicClassification,allNames,isAcquired,revenue,competitors,allIds_DiffbotUsage,...,boardMembers,allUris,quarterlyRevenues,nbLocations,crunchbaseUri,industries,allDescriptions,location,locations,subsidiaries
0,NaN,45,Organization,NaN,NaN,"[Lucent, Lucent Technologies, Inc., 루슨트 테크놀로지,...",NaN,"{'currency': 'USD', 'value': 9440000000.0}","[{'summary': 'American technology company', 'i...","[crbs_c39109, hoovers_sentinel2099178, hoovers...",...,"[{'summary': 'Dutch politician', 'image': 'htt...",[crunchbase.com/organization/lucent-technologi...,NaN,3,crunchbase.com/organization/lucent-technologies,"[Telecommunications Companies, Media And Infor...","[Lucent Technologies, Inc., was an American mu...",{'country': {'summary': 'Sovereign state in No...,[{'country': {'summary': 'Sovereign state in N...,"[{'summary': 'Organization based in Needham, M..."
1,twitter.com/Tillys,562,Organization,NaN,NaN,"[Tillys, TILLY'S, INC., TILLYS INC, Tilly's, T...",NaN,"{'currency': 'USD', 'value': 598478000.0}",NaN,"[SECGOV_10285, STCK_8026, crbs_c1216123, crpwt...",...,"[{'summary': 'Chief Officer at TILLY S INC.', ...","[old.nasdaq.com/symbol/tlys, crunchbase.com/or...","[{'revenue': {'currency': 'USD', 'value': 1631...",4,crunchbase.com/organization/tillys,"[Clothing Retailers, Retailers, Clothing Compa...","[American retail clothing company, Tillys (ori...",{'country': {'summary': 'Sovereign state in No...,[{'country': {'summary': 'Sovereign state in N...,"[{'summary': 'Organization based in Clovis, Ca..."
2,twitter.com/MIPSGuru,34,Organization,"[{'code': '2611', 'isPrimary': False, 'name': ...","[{'code': '2610', 'isPrimary': False, 'name': ...","[MIPS TECHNOLOGIES INC, MIPS Technologies, Inc...",1.0,"{'currency': 'USD', 'value': 86231000.0}",[{'summary': 'Manufacturing company based in U...,"[SECGOV_11763, angl_sentinel6632945, angl_sent...",...,NaN,"[angel.co/mips-computer-systems, angel.co/mips...","[{'revenue': {'currency': 'USD', 'value': 1721...",5,crunchbase.com/organization/mips-technologies,"[Semiconductor Companies, Manufacturing Compan...","[Company, MIPS Technologies, Inc., formerly MI...",{'country': {'summary': 'Sovereign state in No...,[{'country': {'summary': 'Sovereign state in N...,[{'summary': 'Manufacturing company based in P...
3,NaN,0,Organization,NaN,NaN,"[LOG ON AMERICA INC, Log On America, Log On Am...",NaN,NaN,NaN,"[bloomberg_sentinel369040, crbs_c617864, crpwt...",...,"[{'recordId': 'Eliz9meDqPBSnnMTGmbHljA@0', 'na...","[bloomberg.com/profile/company/LOAX:US, bloomb...",NaN,2,crunchbase.com/organization/log-on-america,"[Data Providers, Software Companies]",[LOA was incorporated in Rhode Island (''LOARI...,{'country': {'summary': 'Sovereign state in No...,[{'country': {'summary': 'Sovereign state in N...,NaN
4,twitter.com/PolyOne,1035,Organization,"[{'code': '2016', 'isPrimary': False, 'name': ...","[{'code': '2013', 'isPrimary': False, 'name': ...","[Avient Corporation, Polyone Corporation, Poly...",NaN,"{'currency': 'USD', 'value': 4818800000.0}","[{'summary': 'Specialty chemicals company', 'i...","[SECGOV_1308, STCK_5507, alibaba_sentinel19460...",...,"[{'summary': 'Board Member at Greif, Inc.', 't...","[old.nasdaq.com/symbol/avnt, swnieto.fm.alibab...","[{'revenue': {'currency': 'USD', 'value': 1162...",5,crunchbase.com/organization/polyone,"[Petrochemical Companies, Chemical Companies]","[Company in Avon Lake, United States, Avient C...",{'country': {'summary': 'Sovereign state in No...,[{'country': {'summary': 'Sovereign state in N...,[{'summary': 'Chemical company based in Itupev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,twitter.com/aventinerei,27,Organization,NaN,NaN,"[Aventine Renewable Energy, AVENTINE RENEWABLE...",True,NaN,NaN,"[crbs_c68298, crpwtch_131259, hoovers_sentinel...",...,"[{'summary': 'CEO at TBC Hold

In [496]:
#save to a csv
com_df.to_csv('500comp.csv', encoding='utf-8', index=False)

In [500]:
combined500=pd.concat([random_ipo500, com_df], axis=1)

In [501]:
#combine the API information with the original ipo dataset
combined500.to_csv('500compinfo.csv', encoding='utf-8', index=False)

## This below is using the API to extract patent data from Patentsview

In [3]:
# Use a simple example of the patents view to test and view the output
import requests
url = 'https://api.patentsview.org/patents/query'
r = requests.post(url,  json = {"q":{"_gte":{"patent_date":"2022-01-04"}},"f":["patent_number","patent_date","inventor_last_name"]} )

In [566]:
#load the dataset for pulling information from patentsview
patentori=pd.read_csv("Cominfo_forpatent.csv")

In [655]:
#create an empty dataframe to store the API given information
pa_df = pd.DataFrame(columns=['company','patents', 'count', 'total_patent_count'])

In [656]:
display(pa_df)

,company,patents,count,total_patent_count


In [657]:
# similar to prior method, this also uses a while loop to find the number of patents for each company 
import requests
url = 'https://api.patentsview.org/patents/query'
i=0
while i<=99: #loop through the hundred comapnies
    company=patentori10.iloc[i].newconm #the information needed (name fo the assignee) for finding the patents 
    year=patentori10.iloc[i].year_IPO #restriction of the time
    year=str(year) #so it can be added to the query request
    companyname=patentori10.iloc[i].conm #to add into the dataframe
    r = requests.post(url,  json ={"q":{"_begins":{"assignee_organization":""+company+""}},"f":["assignee_organization","patent_number","patent_title","patent_date"]})
    r_data=json.loads(r.text) #turn into dict type output
    pa_info=[] #patent information first stored in a list
    pa_info=list(r_data.values()) #add the values of the dictionary to the empty list
    pa_info.insert(0,companyname) 
    pa_df.loc[len(pa_df)] = pa_info #append each list as the new row in the dataframe
    i+=1
    

In [658]:
pa_df

,company,patents,count,total_patent_count
0,LUCENT TECHNOLOGIES INC,"[{'patent_number': '5651051', 'patent_title': ...",6,6
1,TILLY'S INC,None,0,0
2,MIPS TECHNOLOGIES INC,None,0,0
3,LOG ON AMERICA INC,None,0,0
4,POLYONE CORP,"[{'patent_number': '10051867', 'patent_title':...",25,136
...,...,...,...,...
95,DATAWATCH CORP,None,0,0
96,SGX PHARMACEUTICALS INC,None,0,0
97,INTERNATIONAL NETWORK SVCS,None,0,0
98,SUNGLASS HUT INTL INC,None,0,0


In [659]:
pa_df.to_csv('Patent_tillnow.csv', encoding='utf-8', index=False)